# Check GPU version.

In [ ]:
!nvidia-smi

# Mount google drive.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# Install TensorFlow-1.14 GPU.

In [1]:
# Select TensorFlow-1.x version.
%tensorflow_version 1.x

# Uninstall previous TensorFlow version.
!pip uninstall tensorflow -y 1>/dev/null 2>/dev/null 
!pip uninstall tensorflow-gpu -y 1>/dev/null 2>/dev/null 

# Install TensorFlow-1.14.
!pip install --upgrade tensorflow==1.14.0 1>/dev/null 2>/dev/null 
!pip install --upgrade tensorflow-gpu==1.14.0 1>/dev/null 2>/dev/null 

TensorFlow 1.x selected.


# Restart the runtime.

# Set the root directory.

In [1]:
import os

root_dir = '/content/'
os.chdir(root_dir)

!ls -al

total 16
drwxr-xr-x 1 root root 4096 Jun 26 16:26 .
drwxr-xr-x 1 root root 4096 Jun 30 11:03 ..
drwxr-xr-x 1 root root 4096 Jun 26 16:26 .config
drwxr-xr-x 1 root root 4096 Jun 26 16:26 sample_data


# Import TensorFlow-1.14.

In [3]:
try:
  %tensorflow_version 1.x
except Exception:
  pass

import tensorflow as tf
from tensorflow.keras import backend as K

import tensorflow.keras.layers as layers
import tensorflow.keras.models as models

import numpy as np
np.random.seed(7)

import matplotlib.pyplot as plot

print(tf.__version__)

1.14.0


# Clone VM workload dataset repository.

In [91]:
!rm -rf VMWorkloadPredictor

In [4]:
!git clone https://github.com/look4pritam/VMWorkloadPredictor.git

Cloning into 'VMWorkloadPredictor'...
remote: Enumerating objects: 2535, done.
remote: Total 2535 (delta 0), reused 0 (delta 0), pack-reused 2535
Receiving objects: 100% (2535/2535), 25.17 MiB | 6.67 MiB/s, done.
Resolving deltas: 100% (92/92), done.
Checking out files: 100% (2524/2524), done.


In [5]:
!ls -al

total 20
drwxr-xr-x 1 root root 4096 Jun 30 11:08 .
drwxr-xr-x 1 root root 4096 Jun 30 11:03 ..
drwxr-xr-x 1 root root 4096 Jun 26 16:26 .config
drwxr-xr-x 1 root root 4096 Jun 26 16:26 sample_data
drwxr-xr-x 7 root root 4096 Jun 30 11:08 VMWorkloadPredictor


In [ ]:
!ls -al VMWorkloadPredictor/output
!ls -al VMWorkloadPredictor/ml_data

# Set dataset root directory.

In [7]:
dataset_path = '/content/VMWorkloadPredictor/ml_data/*.csv'

# Load dataset.

In [8]:
import glob
import csv

In [9]:
sample_history = 10

def load_dataset(dataset_path):
    data = []
    labels = []

    print('Load dataset - Start.')

    for fname in glob.glob(dataset_path):
        with open(fname, 'r') as infh:
            reader = csv.reader(infh, delimiter=';')

            for row in reader:
                r = np.array(row, dtype = float)
                rr = []
                for i in range(sample_history):
                    rr.append(r[i * 7 + 1])
                    rr.append(r[i * 7 + 2])
                #print(rr)
                data.append(rr)
                labels.append(r[-1])
 
    data = np.array(data)
    labels = np.array(labels)
    n = int(float(data.shape[0]) * 0.8)
    train_data = data[:n]
    train_labels = labels[:n]
    test_data = data[n:]
    test_labels = labels[n:]
    print('Load dataset - End.')
    return train_data, train_labels, test_data, test_labels

In [10]:
train_data, train_labels, test_data, test_labels = load_dataset(dataset_path)

Load dataset - Start.
Load dataset - End.


In [11]:
num_classes = 20

In [ ]:
model = models.Sequential()
model.add(layers.Flatten(input_shape=(20,)))
model.add(layers.Dense(units=100, activation='relu'))
model.add(layers.Dense(units=num_classes, activation='softmax'))

In [13]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten (Flatten)            (None, 20)                0         
_________________________________________________________________
dense (Dense)                (None, 100)               2100      
_________________________________________________________________
dense_1 (Dense)              (None, 20)                2020      
Total params: 4,120
Trainable params: 4,120
Non-trainable params: 0
_________________________________________________________________


In [146]:
model.compile(loss='sparse_categorical_crossentropy', optimizer=tf.keras.optimizers.Adam(0.001), metrics=['accuracy'])

In [14]:
model.compile(loss='sparse_categorical_crossentropy', optimizer=tf.keras.optimizers.Nadam(0.001), metrics=['accuracy'])

In [15]:
reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(
    monitor='val_loss', 
    factor=0.1, 
    patience=3, 
    min_delta=0.001,
    min_lr=1e-7, 
    verbose=1)

In [16]:
early_stop = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss', 
    min_delta=0, 
    patience=5, 
    restore_best_weights=True, 
    verbose=1)

In [17]:
epochs = 200

In [18]:
model.fit(
    train_data, train_labels,
    epochs=epochs, 
    callbacks=[reduce_lr, early_stop],
    validation_data=(test_data, test_labels))

Train on 288941 samples, validate on 72236 samples
Epoch 1/200
288941/288941 [==============================] - 13s 46us/sample - loss: 1.0443 - acc: 0.7320 - val_loss: 0.6627 - val_acc: 0.8171
Epoch 2/200
288941/288941 [==============================] - 13s 46us/sample - loss: 0.6648 - acc: 0.8174 - val_loss: 0.5350 - val_acc: 0.8433
Epoch 3/200
288941/288941 [==============================] - 14s 48us/sample - loss: 0.6085 - acc: 0.8350 - val_loss: 0.4987 - val_acc: 0.8499
Epoch 4/200
288941/288941 [==============================] - 13s 46us/sample - loss: 0.5673 - acc: 0.8423 - val_loss: 0.4995 - val_acc: 0.8513
Epoch 5/200
288941/288941 [==============================] - 13s 46us/sample - loss: 0.5313 - acc: 0.8473 - val_loss: 0.4944 - val_acc: 0.8554
Epoch 6/200
288941/288941 [==============================] - 13s 47us/sample - loss: 0.5246 - acc: 0.8508 - val_loss: 0.4903 - val_acc: 0.8559
Epoch 7/200
288941/288941 [==============================] - 13s 46us/sample - loss: 0.5280

In [34]:
!python preprocess.py

287
287
287
287
287
579
287
286
287
287
288
287
287
287
287
287
287
287
287
538
287
287
287
287
287
287
287
287
287
287
287
58
287
287
287
287
287
64
675
287
376
287
287
287
287
287
141
287
287
287
287
287
287
288
287
287
579
287
287
376
287
287
287
287
287
287
287
287
287
287
287
287
287
287
287
276
287
287
287
220
287
0
287
287
287
287
287
287
288
287
287
287
287
287
537
287
287
287
287
287
249
287
287
287
287
287
58
287
218
287
287
287
287
287
287
287
287
287
287
287
287
287
287
202
287
288
287
287
287
789
287
287
287
287
287
287
287
287
376
287
287
287
249
287
287
287
287
288
287
287
287
287
287
287
287
345
287
249
287
287
287
287
287
579
287
376
287
287
288
287
287
311
211
287
578
287
287
287
287
287
287
287
287
287
287
287
287
69
288
287
287
287
287
287
287
287
287
675
287
278
287
287
287
287
287
287
287
287
287
287
287
287
287
287
376
287
287
579
287
287
287
287
287
278
287
287
287
287
287
287
287
287
287
287
287
287
249
287
675
287
287
578
126
268
287
287
288
287
287
57
287
287

In [4]:
!python main_cpu.py

loading data...
finished loading data
/usr/local/lib/python3.6/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
Best: MLPClassifier(activation='relu', alpha=0.01, batch_size='auto', beta_1=0.9,
              beta_2=0.999, early_stopping=False, epsilon=1e-08,
              hidden_layer_sizes=(100,), learning_rate='constant',
              learning_rate_init=0.001, max_fun=15000, max_iter=200,
              momentum=0.9, n_iter_no_change=10, nesterovs_momentum=True,
              power_t=0.5, random_state=None, shuffle=True, solver='adam',
              tol=0.0001, vali